# Librerías

In [209]:
import arcpy
import random
import numpy as np, numpy.random
from numpy.random import multinomial

# Funciones

In [210]:
def crear_dict_uso():
    """
    Genera un diccionario que contiene los distintos tipos de uso de suelo y sus respectivos 
    totales de superficie a distribuir dentro de la capa de manzanas.

    Returns:
        dict_uso_suelo (dict): Diccionario de usos de suelo y su porcentaje ingresado por el usuario.

    """
     
    # Se inicializa el diccionario
    dict_uso_suelo = {"residencia": 0,
                      "comercio": 0,
                      "educacion": 0,
                      "salud": 0,
                      "oficinas": 0,
                      "parque":0}

    # Ciclo para solicitar porcentaje de cada uso de suelo
    for uso_suelo in dict_uso_suelo:

        print("Ingrese la superficie total de uso de suelo tipo " + uso_suelo)
        dict_uso_suelo[uso_suelo] = int(input())
        
    return dict_uso_suelo

In [211]:
def crear_dict_gse():
    """
    Genera diccionario de los distintos GSE's para las zonas:
    
    Returns:
        dict_gse (dict): Diccionario de los GSE a incluir dentro del Feature Class.
        
    """
    
    # Se inicializa el diccionario
    dict_gse = {"p_bajo": 0,
                "p_medio": 0,
                "p_alto": 0}
    
    return dict_gse

In [212]:
def contar_registros_fc(feature_class):
    """
    Cuenta el número de filas de un determinado feature class.
    
    Args:
        feature_class (str): Nombre de la feature class cargada en el proyecto.
        
    Returns:
        n_rows (int): Número de filas del Feature Class.
    """
    n_rows = int(arcpy.GetCount_management("manzanas").getOutput(0))
    
    return n_rows

In [231]:
def crear_np_gse(n_rows):
    """
    Genera un numpy array de 3 dimensiones, donde la suma total de cada fila es igual a 100.
    Cada columna representa un GSE.
    
    Args:
        n_rows (int): N° de filas del Feature Class al que se le añadirán las columnas
        
    Returns:
        np_gse (np.array): Numpy array de 3 dimensiones que representa la distribución de GSE's de cada zona.
    """
    
    np_gse = np.vstack([(np.random.dirichlet(np.ones(3),size=1)*100)[0] for i in range(n_rows)])
    
    return np_gse



In [214]:
def crear_lista_random(suma_total, n_rows):
    """
    Crea una lista de tamaño n_rows donde la suma corresponde a suma_total. 
    Los valores se distribuyen según una distribución binomial.
    
    Args:
        suma_total (int): Suma total de la lista generada.
        n_rows (int): Número de elementos de lista a generar.
        
    Returns:
        lista_random (list): Lista de tamaño n_rows y suma equivalente a suma_total
    """
    
    lista_random = multinomial(suma_total, [1/n_rows] * n_rows)
    
    return lista_random

In [215]:
def crear_lista_uso(dict_ptjes, n_rows):
    """
    Genera una lista del tamaño de filas de la capa de manzanas que contiene tipos de uso de
    suelo repartida de acuerdo a los porcentajes establecidos inicialmente
    
    Args:
        dict_ptjes (dict): Diccionario de usos de suelo y sus respectivos porcentajes
        n_rows (int): Número de filas del Feature Class
        
    Returns:
        lista_uso_suelos (list): Lista de tamaño n_rows que contiene distintos usos de suelo. 
    """
    
    lista_uso_suelos = sum([[key]*int(dict_ptjes[key]*(n_rows/100)) for key in dict_ptjes],[])
    
    # Se desordena la lista
    random.shuffle(lista_uso_suelos)
    
    return lista_uso_suelos

In [216]:
def get_columnas(feature_class):
    """
    Devuelve la lista de columnas de un feature_class
    
    Args:
        feature_class (str): Nombre del feature_class
        
    Returns:
        lista_columnas (list): Lista de columnas
    """
    lista_columnas = [columna.name for columna in arcpy.ListFields(feature_class)]
    return lista_columnas

In [217]:
def eliminar_columna(feature_class, columna, lista_columnas):
    """
    Elimina una columna de un determinado feature class.
    
    Args:
        feature_class (str): Nombre del Feature Class al que se elimina la columna
        columna (str): Nombre de la columna a eliminar
        lista_columnas (list): Lista de columnas del feature class
    """
    
    if columna in lista_columnas:
        arcpy.DeleteField_management (feature_class, columna)

In [229]:
def crear_columna(feature_class, nombre_columna, tipo_columna, largo_columna = None):
    """
    Genera una nueva columna en el feature class indicado.
    
    Args:
        feature_class (str): Nombre del feature class al que se añade la columna
        nombre_columna (str): Nombre de la columna a añadir al feature class
        tipo_columna (str): Tipo de dato a alojar en la nueva columna
        largo_columna (int): Largo del dato a alojar en la nueva columna
    """
    arcpy.AddField_management(feature_class,
                              nombre_columna,
                              tipo_columna,
                              largo_columna)

In [219]:
def crear_cursor(feature_class):
    """
    Genera un cursor para iterar sobre las filas del feature class.
    
    Args:
        feature_class (str): Nombre del feature class al cual se le quiere generar el cursor.
    
    Returns:
        cursor (arcpy.arcobjects.arcobjects.Cursor): Cursor del FC indicado.
    """
    cursor = arcpy.UpdateCursor(feature_class)
    
    return cursor

In [220]:
def lista_a_columna(feature_class, lista_columna, nombre_columna):
    """
    Transforma una lista en una columna de un feature class.
    
    Args:
        feature_class (str): Nombre del feature class a añadirle la columna.
        lista_columna (list): Lista que contiene los valores a ingresar en la columna.
        nombre_columna (str): Nombre de la columna a la que se le ingresarán valores.
    """

    with arcpy.da.UpdateCursor(feature_class, nombre_columna) as cursor:
        for i, row in enumerate(cursor):
            row[0] = lista_columna[i]
            cursor.updateRow(row)

# Main

In [232]:
# Quizá alguna función para cargar la capa dentro

# Se solicita al usuario ingresar la superficie total de cada uso de suelo
dict_superficie = crear_dict_uso()

# Se genera el diccionario de GSE's
dict_gse = crear_dict_gse()

# Se cuenta el número de filas de la capa de manzanas
n_rows = contar_registros_fc("manzanas")

# Se crean las listas de acuerdo a los valores ingresados inicialmente

lista_residencial = crear_lista_random(dict_superficie['residencia'], n_rows)
lista_comercio = crear_lista_random(dict_superficie['comercio'], n_rows)
lista_educacion = crear_lista_random(dict_superficie['educacion'], n_rows)
lista_salud = crear_lista_random(dict_superficie['salud'], n_rows)
lista_oficinas = crear_lista_random(dict_superficie['oficinas'], n_rows)
lista_parque = crear_lista_random(dict_superficie['parque'], n_rows)

# Se crea una lista de columnas del feature class
lista_columnas = get_columnas("manzanas")

# Se eliminan columnas y se crean de nuevo

## Columnas de tipos de uso
for uso in dict_superficie:
    
    eliminar_columna("manzanas", uso, lista_columnas)
    crear_columna("manzanas", uso, "SHORT", 50)

# Se transforma la lista a una columna del Feature Class de manzanas original
lista_a_columna("manzanas", lista_residencial, "residencia")
lista_a_columna("manzanas", lista_comercio, "comercio")
lista_a_columna("manzanas", lista_educacion, "educacion")
lista_a_columna("manzanas", lista_salud, "salud")
lista_a_columna("manzanas", lista_oficinas, "oficinas")
lista_a_columna("manzanas", lista_parque, "parque")

## Columnas GSE
for gse in dict_gse:
    
    eliminar_columna("manzanas", gse, lista_columnas)
    crear_columna("manzanas", gse, "FLOAT")

# Se genera el array de porcentaje de distribución de los distintos GSE's en las zonas
np_gse = crear_np_gse(n_rows)
    
# Listas de distintos GSE's
gse_bajo = np_gse[:,0]
gse_medio = np_gse[:,1]
gse_alto = np_gse[:,2]

# Se transforman las listas de GSE's a columnas 
lista_a_columna("manzanas", gse_bajo, "p_bajo")
lista_a_columna("manzanas", gse_medio, "p_medio")
lista_a_columna("manzanas", gse_alto, "p_alto")

# Pasar a GDB

Ingrese la superficie total de uso de suelo tipo residencia
10000
Ingrese la superficie total de uso de suelo tipo comercio
10000
Ingrese la superficie total de uso de suelo tipo educacion
10000
Ingrese la superficie total de uso de suelo tipo salud
100000
Ingrese la superficie total de uso de suelo tipo oficinas
10000
Ingrese la superficie total de uso de suelo tipo parque
10000
